<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_teachability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chatting with TeachableAgent

Conversational assistants based on LLMs can remember the current chat with the user, and can even demonstrate in-context learning of things that the user teaches the assistant during the chat. But these memories and learnings are lost once the chat is over, or when a single chat grows too long for the LLM to handle effectively. In subsequent chats, the user is forced to repeat any necessary instructions over and over.

`TeachableAgent` addresses these limitations by persisting user teachings across chat boundaries in long-term memory (a vector database). Memory is saved to disk at the end of each chat, then loaded from disk at the start of the next. Instead of copying all of memory into the context window, which would eat up valuable space, individual memories (called memos) are retrieved into context as needed. This allows the user to teach frequently used facts and skills to the agent just once, and have the agent remember them in later chats.

In making decisions about memo storage and retrieval, `TeachableAgent` calls an instance of `TextAnalyzerAgent` to analyze pieces of text in several different ways. This adds extra LLM calls involving a relatively small number of tokens. These calls can add a few seconds to the time a user waits for a response.

This notebook demonstrates how `TeachableAgent` can learn facts, preferences, and skills from users. To chat with `TeachableAgent` yourself, run [chat_with_teachable_agent.py](../test/agentchat/chat_with_teachable_agent.py).

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install the [teachable] option.
```bash
pip install "pyautogen[teachable]"
```

In [1]:
%%capture --no-stderr
# %pip install "pyautogen[teachable]

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [2]:
import autogen

config_list = autogen.config_list_from_json(
    env_or_file="OAI_CONFIG_LIST",
    file_location="",
    filter_dict={
        "model": ["gpt-4"],
    },
)

print(config_list[0]["model"])

gpt-4


It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST". It filters the configs by models (you can filter by other keys as well). After application of this particular filter, only the gpt-4 models are kept.

The config list looks like the following:
```python
config_list = [
    {
        'model': 'gpt-4',
        'api_key': '<your OpenAI API key here>',
    },
    {
        'model': 'gpt-4',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
    {
        'model': 'gpt-4-32k',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
]
```

If you open this notebook in colab, you can upload your files by clicking the file icon on the left panel and then choose "upload file" icon.

You can set the value of config_list in other ways if you prefer, e.g., loading from a YAML file.

## Construct Agents
For this walkthrough, we start by resetting the agent's memory store. This deletes any memories from prior conversations that may be stored on disk.

In [3]:
from autogen.agentchat.contrib.teachable_agent import TeachableAgent
from autogen import UserProxyAgent, AssistantAgent

llm_config = {
    "request_timeout": 1000,
    "config_list": config_list,
}

teach_config={
    "verbosity": 1,  # 0 for basic info, 1 to add memory operations, 2 for analyzer messages, 3 for memo lists.
    "reset_db": True,  # Set to True to start over with an empty database.
    "path_to_db_dir": "./tmp/notebook/teachable_agent_db",  # Path to the directory where the database will be stored.
    "recall_threshold": 1.5,  # Higher numbers allow more (but less relevant) memos to be recalled.
}

try:
    from termcolor import colored
except ImportError:
    def colored(x, *args, **kwargs):
        return x
    
autogen.ChatCompletion.start_logging()

agent = TeachableAgent(
    name="agent",
    llm_config=llm_config,
    teach_config=teach_config)

user = UserProxyAgent(
    name="user",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: True if "TERMINATE" in x.get("content") else False,
    max_consecutive_auto_reply=0,        
)




CLEARING MEMORY


## Learning new facts
Let's teach the agent some facts it doesn't already know, since they are more recent than GPT-4's training data.

In [4]:
text = "Hi can you please explain fractions to me?"
user.initiate_chat(agent, message=text, clear_history=True)

user (to agent):

Hi can you please explain fractions to me?

--------------------------------------------------------------------------------
agent (to user):

Absolutely, I'd be happy to explain fractions.

A fraction is a method of expressing a quantity by using a numerator and a denominator. It represents a part of a whole.

1. The **numerator** is the number above the line or before the slash in a fraction. It represents the number of parts we have.

2. The **denominator** is the number below the line or after the slash. It represents the total number of equal parts into which the whole is divided.

For example, in the fraction 3/4, 

- 3 is the numerator, which means we have 3 parts
- 4 is the denominator, which means the whole is divided into 4 parts

So, 3/4 means we have 3 parts of a whole which is divided into 4 equal parts.

A fraction can also be seen as a division operation – the fraction 3/4 represents the division of 3 by 4, which yields 0.75 in decimal form. 

Some frac

In [5]:
text = "I don't understand this! Actually I would understand math operators better if you could explain it using money"
user.initiate_chat(agent, message=text, clear_history=False)

user (to agent):

I don't understand this! Actually I would understand math operators better if you could explain it using money

--------------------------------------------------------------------------------
agent (to user):

Absolutely, using real world examples like money can often make understanding fractions easier.

Let's stick with the fraction 3/4 and think about it in terms of dollars (or any other unit of currency you prefer).

1. Imagine you have a whole dollar. This whole dollar represents 1 in fraction terms.
2. Now, let's say you have to divide this dollar evenly into 4 parts, because the denominator of our fraction is 4. Each of these parts would be 25 cents, because a dollar consists of 100 cents and 100 divided by 4 is 25.
3. The fraction we are working with is 3/4, where 3 is the numerator, so we take 3 parts out of these 4 parts each worth 25 cents. So, we now have 75 cents.

Thus, the fraction 3/4 in terms of money would be 75 cents of a whole dollar. Another way 

In [6]:
agent.learn_from_user_feedback()


REVIEWING CHAT FOR USER TEACHINGS TO REMEMBER

INPUT-OUTPUT PAIR ADDED TO VECTOR DATABASE:
  INPUT
    Can you explain math operators to me using money? I think it'd help me understand better.
  OUTPUT
    I don't understand this! Actually I would understand math operators better if you could explain it using money


# New Chat Session
Let's end our first chat here. The following function needs to be called at the end of each chat, so that `TeachableAgent` can store what the user has taught it.

In [7]:
text = "Hi can you please explain exclusive OR to me?"
user.initiate_chat(agent, message=text, clear_history=True)

user (to agent):

Hi can you please explain exclusive OR to me?

--------------------------------------------------------------------------------

MEMOS APPENDED TO LAST USER MESSAGE...

# Memories that might help
- I don't understand this! Actually I would understand math operators better if you could explain it using money


agent (to user):

Certainly, I can explain the concept of Exclusive OR (XOR) using a money-based analogy.

So, let's imagine you have two friends, Alice and Bob. You have a rule that you will go out for a pizza only if one, and only one of your friends, pays for it: either Alice or Bob - not both, and not neither of them.

This is the principle of Exclusive OR (XOR). In a simple two-input XOR logic:

- If Alice pays (True) and Bob doesn't (False), you can go out for pizza. (True)
- If Alice doesn't pay (False) and Bob pays (True), you still can go out for pizza. (True)
- If neither Alice nor Bob pays (both are False), you can't go out for a pizza. (False) 
- If b

Now let's start a new chat by clearing the previous chat's history. At this point, common LLM-based assistants would forget everything from the last chat. But `TeachableAgent` can retrieve memories from its vector DB as needed, allowing it to recall and reason over facts that the user taught it in earlier conversations.